In [1]:
import numpy as np
import pandas as pd
from keras import layers
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split

import pathlib
import cv2
import os
import shutil
import re
import time

import random

## Plan 

1) division des données

2) generation d'images via gan

3) 1er modele et resultat

4) amélioration de la qualité des images via vae

5) 1er modèle entrainer sur les images améliorer

6) Prédiction avec YOLO

## Division des données

Nous disposant de 4 dossiers contenant chacune une catégorie d'image.
* MildDemented contient 896 fichier
* ModerateDemented:64 fichier
* NonDemented: 3200 fichier
* VeryMildDemented: 2240 fichier

On remarque que les classes sont désequilibré notamment pour la catégorie ModerateDemented qui n'a que 64 images. 


Commençons par scindé ces dossiers en 3 dossiers afin d'avoir un jeu de donnée pour les étapes de validation, d'entrainement et de test.

In [ ]:

# On renomme les images afin de facilité la réorganisation
path="C:/Users/ranus/alzheimer/archive(1)/OriginalDataset"
folder_path=[]
cat=[]
for dirname,_, filename in os.walk(path):
    folder_path.append(dirname)
    cat.append(re.sub("\\\\","/",dirname).split('/')[-1])


del cat[0]    
del folder_path[0] #On retire le 1er élément qui correspond au chemin path

for i in range(len(folder_path)):
    folder_path[i]=re.sub("\\\\","/",folder_path[i]) #On corrige les antislash en slash

    
#On renomme les fichiers
k=0 #Accumulateur pour parcourir la liste des labels
for pat in folder_path:
    j=0 #initalisationde l'accumulateur
    for dirname ,_, filenames  in os.walk(pat):
        for file in filenames:
            a=re.sub("\\\\","/",os.path.join(pat,file))
            b=re.sub("\\\\","/",os.path.join(pat,cat[k]+str(j)+'.jpg'))
            #os.rename(a,b)
            j+=1
    k+=1

#Créations de liste contenant les différents composante des futurs chemin d'accés
l=[]
cat=[]
for dirname ,_, filenames  in os.walk(path):
    cat.append(dirname.split("\\")[-1])
    l.append(dirname)
del cat[0]
a=["train_dir","validation_dir","test_dir"]


#Renommons toutes les fichier de chaque dossier afin de faciliter le copie.


#Création des dossier vide et copie des images
path2=re.sub("\\\\","/",os.path.join(path,"SplitData"))
os.mkdir(path2)

for A in a:
    os.mkdir(re.sub("\\\\",'/',os.path.join(path2,A)))

for i in cat:
    p0=re.sub("\\\\","/",os.path.join(path,i))
    n=len(os.listdir(p0))#compte le nombre d'element du fichier i
    fnames=os.listdir(p0)
    for j in a:
        p=re.sub("\\\\","/",os.path.join(path2,j,i))
        os.mkdir(str(p))
            #fnames=['{}.jpg' for k in range(0,2*n//4)]# 50% des données serviront à l'entrainement.
        if j=="train_dir":
            for k in range(0,2*n//4):
            #for fname in fnames:
                src=re.sub("\\\\","/",os.path.join(p0,fnames[k]))
                dst=re.sub("\\\\","/",os.path.join(p,i+str(k)+".jpg"))
                os.renames(src,dst)
        elif j=='validation_dir':
            for k in range(2*n//4,3*n//4):
           # fnames=['{}.jpg' for k in range(2*n//4,3*n//4)]# 25% des données serviront à la validation.
            #for fname in fnames:
                src=re.sub("\\\\","/",os.path.join(p0,fnames[k]))
                dst=re.sub("\\\\","/",os.path.join(p,i+str(k)+".jpg"))
                os.renames(src,dst)
        elif j=='test_dir':
        #fnames=['{}.jpg' for k in range(3*n//4,n)]# 25% des données serviront à tester le modèle.
       # for fname in fnames:
            for k in range(3*n//4,n):
                src=re.sub("\\\\","/",os.path.join(p0,fnames[k]))
                dst=re.sub("\\\\","/",os.path.join(p,i+str(k)+".jpg"))
                os.renames(src,dst)

## Generation d'image via un réseau GAN

Afin de résoudre le problème des classes déséquilibrés souligné plus tôt, on va créer des images à l'aide d'un réseau GAN.
De plus pour éviter de créer des images appartenant aux autres catégorie nous les utiliserons pour entrainer le discriminant.

In [ ]:
path="C:/Users/ranus/alzheimer/archive(1)/OriginalDataset"
liste=[]
for dirname,_,filenames in os.walk(path):
    #if dirname=='ModerateDemented':
    for file in filenames:
        liste+=[re.sub("\\\\","/",os.path.join(dirname,file))]
im=cv2.imread(liste[0])
plt.imshow(im)
im.shape


In [ ]:
def generateur_model():
    model=tf.keras.Sequential()
    model.add(layers.Dense(208*176*3, use_bias=False, input_shape=[208,176,3])) #initialisation d'un vecteur bruit
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Reshape((208,176,3)))
    
    model.add(layers.Conv2DTranspose(512,(5,5),strides=(2,2),padding='same',use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    model.add(layers.Conv2DTranspose(256,(5,5),strides=(2,2),padding='same',use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    model.add(layers.Conv2DTranspose(128),(5,5),strides=(2,2),padding="same",use_bias=False)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    model.add(layers.Conv2DTranspose(64),(5,5),strides=(2,2),padding="same",use_bias=False)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    model.add(layers.Conv2DTranspose(1,(5,5),strides=(2,2),padding='same',use_bias=False,activation='tanh'))
    
    return model

def discriminant_model():
    model=tf.keras.Sequential()
  
    model.add(layers.Dense(4*4*1024, use_bias=False, input_shape=[208,176,3]))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLu())
    
   
    model.add(layers.Conv2D(256,(5,5),stides(2,2),padding='same')
    model.add(layers.LeakyRelu()) 
    
    model.add(layers.Conv2D(128,(5,5),stides(2,2),padding='same')
    model.add(layers.LeakyRelu()) 
    
    model.add(layers.Conv2D(64,(5,5),stides(2,2),padding='same')
    model.add(layers.LeakyRelu()) 
    model.add(layers.Dense(1,activation'sigmoid'))

## Pré traitement des données

Convertissons les images en tenseur.

In [24]:
train_dir="C:/Users/ranus/alzheimer/archive(1)/OriginalDataset/SplitData/train_dir"
valid_dir="C:/Users/ranus/alzheimer/archive(1)/OriginalDataset/SplitData/validation_dir"
test_dir="C:/Users/ranus/alzheimer/archive(1)/OriginalDataset/SplitData/test_dir"

train_datagen=ImageDataGenerator(rescale=1./255)
test_datagen=ImageDataGenerator(rescale=1./255)
valid_datagen=ImageDataGenerator(rescale=1./255)

train_generator=train_datagen.flow_from_directory(train_dir,target_size=(208,176),batch_size=20,class_mode='categorical')
test_generator=test_datagen.flow_from_directory(test_dir,target_size=(208,176),batch_size=20,class_mode='categorical')
valid_generator=valid_datagen.flow_from_directory(valid_dir,target_size=(208,176),batch_size=20,class_mode='categorical')

Found 3173 images belonging to 4 classes.
Found 1587 images belonging to 4 classes.
Found 1587 images belonging to 4 classes.


On a bien le bon nombre de label.

## Classification

### Modele de CNN

In [25]:
model=tf.keras.models.Sequential()

model.add(layers.Conv2D(32,(3,3),activation="relu",input_shape=(208,176,3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(layers.Dropout(0.2))

model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128,(3,3),activation='relu'))
model.add(layers.Dropout(0.2))

model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128,(3,3),activation='relu'))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Flatten())
model.add(layers.Dense(512,activation='relu'))
model.add(layers.Dense(4,activation='softmax'))

model.build(input_shape=(208, 176, 3))
model.summary()

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=["acc"])

#tensorboard = tf.keras.callbacks.TensorBoard(log_dir="logs/{}".format(time.time()))

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_42 (Conv2D)          (None, 206, 174, 32)      896       
                                                                 
 max_pooling2d_41 (MaxPoolin  (None, 103, 87, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_43 (Conv2D)          (None, 101, 85, 64)       18496     
                                                                 
 dropout_20 (Dropout)        (None, 101, 85, 64)       0         
                                                                 
 max_pooling2d_42 (MaxPoolin  (None, 50, 42, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_44 (Conv2D)          (None, 48, 40, 128)     

In [26]:
history=model.fit(train_generator,validation_data=valid_generator)

InvalidArgumentError: Graph execution error:

Detected at node 'sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits' defined at (most recent call last):
    File "C:\Users\ranus\anaconda3\envs\Tensorflow\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\ranus\anaconda3\envs\Tensorflow\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\ranus\anaconda3\envs\Tensorflow\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "C:\Users\ranus\anaconda3\envs\Tensorflow\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Users\ranus\anaconda3\envs\Tensorflow\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "C:\Users\ranus\anaconda3\envs\Tensorflow\lib\site-packages\tornado\platform\asyncio

In [ ]:
#path2="C:/Users/ranus/alzheimer/archive(1)/OriginalDataset"
#image=[]
#categorie=[]
#for dirname, _,filenames in os.walk(path2):
 #   for filename in filenames:
  #      image+=[os.path.join(dirname, filename)]
   #     categorie+=[dirname.split("\\")[-1]]

#df=pd.DataFrame({"images":image,"categories":categorie})
#df=df.sample(frac=1)
#df
#df
#np.random.shuffle(df)

In [ ]:
df["categories"].value_counts()

### Traitement des données

In [ ]:
#from keras.preprocessing.image import ImageDataGenerator

#train_datagen=ImageDataGenerator(rescale=1./255)
#test_datagen=ImageDataGenerator(rescale=1./255)
data_test
def traitement(data):
    for i in range(0,data.shape[0]-1):
        images+=[cv2.imread(data.iloc[i:0])/255]
        labels+=[data.iloc[i:1]]

In [ ]:



path="C:/Users/ranus/alzheimer/archive(1)/OriginalDataset"
folder_path=[]
label=[]
for dirname,_, filename in os.walk(path):
    folder_path.append(dirname)
    label.append(dirname.split('/')[-1])
    
del folder_path[0] #On retire le 1er élément qui correspond au chemin path

for i in range(len(folder_path)):
    folder_path[i]=re.sub("\\\\","/",folder_path[i]) #On corrige les antislash en slash

    
#On renomme les fichiers
for pat in folder_path:
    j=0 #initalisationde l'accumulateur
    for dirname ,_, filenames  in os.walk(pat):
        a=re.sub("\\\\","/",os.path.join(pat,filenames[j]))
        b=re.sub("\\\\","/",os.path.join(pat.split("/")[-1],str(j)+'.jpg'))
        j+=1
        #print(a)
        #print(b)

In [ ]:
path="C:/Users/ranus/alzheimer/archive(1)/OriginalDataset"
folder_path=[]
for dirname,_, filename in os.walk(path):
    folder_path.append(dirname)
    
del folder_path[0] #On retire le 1er élément qui correspond au chemin path

for i in range(len(folder_path)):
    folder_path[i]=re.sub("\\\\","/",folder_path[i]) #On corrige les antislash en slash

    
#On renomme les fichiers
for pat in folder_path:
    j=0 #initalisationde l'accumulateur
    for dirname ,_, filenames  in os.walk(pat):
        for file in filenames:
            a=os.path.join(pat,file)
            print(re.sub("\\\\","/",a))

## YOLO

In [ ]:
os.listdir(p0)[1]

re.sub("\\\\","/",os.path.join(p0,fnames[k]))
re.sub("\\\\","/",os.path.join(p,str(k)+".jpg"))
cat